In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rv_histogram, rv_discrete, binom
from statsmodels.distributions.empirical_distribution import ECDF

### Entradas conocidas

In [57]:
datos = pd.read_excel('Datos/Ejercicio_32.xlsx')
trials = 5000

In [7]:
reservations = datos["Reservations"].values
probabilities = datos["Probability"].values

### Variables de decisión

In [43]:
seed = 1234
ingreso_c = [2000, 2000, 1500, 1500]
prob_c = [0.25, 0.25, 0.4, 0.4]
min_reunion_c = [3, 1, 3, 1]

### Entradas inciertas

In [59]:
def gen_reserva():
    return rv_discrete(name='c_reservas', values=(reservations, probabilities)).rvs(size=1)[0]

print(gen_reserva())

3


### Cálculos intermedios

In [61]:
def simular(ingreso_c, prob_c, min_reunion_c):
    nro_reservas = gen_reserva()
    if nro_reservas < min_reunion_c:
        nro_cirugias = 0
    else:
        nro_cirugias = binom.rvs(n=nro_reservas, p=prob_c, size=1)[0]
    ingreso_c = nro_cirugias*ingreso_c
    
    return ingreso_c

### Salidas

In [67]:
resultados = {}
for ingreso, prob, reunion in zip(ingreso_c, prob_c, min_reunion_c):
    vector_utilidades = []
    for i in range(trials):
        vector_utilidades.append(simular(ingreso, prob, reunion))
    key = f'Ingreso: {ingreso}, Prob: {prob}, Reunion: {reunion}'
    resultados[key] = vector_utilidades
    print(f'Ingreso: {ingreso}, Prob: {prob}, Reunion: {reunion}, {np.mean(vector_utilidades)}')

Ingreso: 2000, Prob: 0.25, Reunion: 3, 2056.0
Ingreso: 2000, Prob: 0.25, Reunion: 1, 2212.0
Ingreso: 1500, Prob: 0.4, Reunion: 3, 2458.8
Ingreso: 1500, Prob: 0.4, Reunion: 1, 2595.0


### Código Tarea

In [ ]:
hist, bin_edges = np.histogram(reservations, bins=len(reservations), weights=probabilities)
dist = rv_histogram((hist, bin_edges))

In [28]:
probabilidad_cirugia = 0.25
costo_cirugia = 2000
costo_cirugia_reducido = 1500
probabilidad_cirugia_reducido = 0.40

In [36]:
asistentes_esperados = np.sum(np.where(reservations > 2, reservations, 0) * probabilities)

print(asistentes_esperados)

4.159999999999999


#### Ingreso esperado con cancelación de sesiones con dos o menos reservas

In [38]:
asistentes_esperados = np.sum(np.where(reservations > 2, reservations, 0) * probabilities)

cirugias_esperadas = probabilidad_cirugia * asistentes_esperados

costo_esperado = cirugias_esperadas * costo_cirugia

print(f"Ingreso esperado por semana: ${costo_esperado:.2f}")

Ingreso esperado por semana: $2080.00


#### Ingreso esperado si no se cancelan las sesiones con dos o menos reservas

In [40]:
reservas_esperadas = dist.mean()

cirugias_esperadas_1 = probabilidad_cirugia * reservas_esperadas

costo_esperado_1 = cirugias_esperadas_1 * costo_cirugia

print(f"Ingreso esperado por semana sin cancelaciones: ${costo_esperado_1:.2f}")

Ingreso esperado por semana sin cancelaciones: $2213.64


#### Ingreso esperado con precio reducido

In [42]:
cirugias_esperadas_2 = probabilidad_cirugia_reducido * asistentes_esperados

costos_esperados_2 = cirugias_esperadas_2 * costo_cirugia_reducido

print(f"Ingreso esperado por semana con precio reducido: ${costos_esperados_2:.2f}")

Ingreso esperado por semana con precio reducido: $2496.00
